In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from tabulate import tabulate

# COBE

In [ ]:
url = 'https://cdn.cboe.com/data/us/futures/market_statistics/historical_data/VX/VX_2024-05-22.csv'
raw_df = pd.read_csv(url)

raw_df.tail()

In [ ]:
vixf = pd.DataFrame({'VIXF':raw_df['Close'].to_list()}, index=pd.to_datetime(raw_df['Trade Date']))
vix = yf.download('^VIX', start=vixf.index[0])
spx = yf.download('^SPX', start=vixf.index[0])

df = pd.concat([spx['Adj Close'].rename('SPX'),
                vix['Adj Close'].rename('VIX'),
                vixf['VIXF'].rename('VIXF')], axis=1)
df = df.dropna()

In [ ]:
spread = df['VIXF'] - df['VIX']
spread.plot()

In [ ]:
def smoothed_rolling_z(raw, window_sma: int, window_z: int):
    smoothed = raw.rolling(window_sma).mean()
    rolling_z = (smoothed - smoothed.rolling(window_z).mean()) / smoothed.rolling(window_z).std()
    return rolling_z

In [ ]:
rolling_z = smoothed_rolling_z(spread, 16, 20)
rolling_z.tail(20)

In [ ]:
from scipy.signal import argrelextrema
def get_local_minimax(ds:pd.Series, order:int) -> pd.Series:
    ds = pd.DataFrame(ds)
    is_maxima_num_index = argrelextrema(ds.to_numpy(), np.greater, axis=0, order=order)[0]
    is_minima_num_index = argrelextrema(ds.to_numpy(), np.less, axis=0, order=order)[0]
    local_max_index_ls = []
    local_min_index_ls = []
    for i in is_maxima_num_index:
        if i > order and i <len(ds) - order:
            local_max_index_ls.append(ds.iloc[i:i+1].index)
    for i in is_minima_num_index:
        if i > order and i <len(ds) - order:
            local_min_index_ls.append(ds.iloc[i:i+1].index)      
    local_max_index_arr = np.array(local_max_index_ls).reshape(-1)
    local_min_index_arr = np.array(local_min_index_ls).reshape(-1)

    this_ds = ds.copy()
    this_ds[f'local minimax'] = 0
    this_ds.loc[local_max_index_arr, 'local minimax'] = -1
    this_ds.loc[local_min_index_arr, 'local minimax'] = 1
    return this_ds[f'local minimax']

In [ ]:
plt.style.use('seaborn-v0_8-white')
fig, ax1 = plt.subplots(figsize=(20, 8), dpi=400)
ax2 = ax1.twinx()

ax1_ds = df['SPX']
ax1.plot(ax1_ds, label='SPX', c='black', alpha=1)
ax1.legend(loc=2, fontsize=14)

minima_ds = rolling_z
order = 10
for _x in minima_ds.loc[(get_local_minimax(minima_ds, order))==1].index:
    ax2.axvline(_x, color='#83c69b', alpha=0.75)   
for _x in minima_ds.loc[(get_local_minimax(minima_ds, order))==-1].index:
    ax2.axvline(_x, color='#eab9d9', alpha=0.75)  

ax2.plot(rolling_z, label='Spread', c='#1f77b4', alpha=0.5)
ax2.legend(loc=1, fontsize=14)

ax2.axhline(0, alpha=0.5, linestyle='--')
ax2.axhline(1.5, alpha=0.5, linestyle='--')
ax2.axhline(-1.5, alpha=0.5, linestyle='--')

plt.title(f'SPX & VIXF-VIX Spread')
plt.show()

In [ ]:
rolling_z.tail(10)

# Investing.hk

In [ ]:
url = 'https://hk.investing.com/indices/us-spx-vix-futures-historical-data'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
data = []

tbody = soup.find_all('tbody')[1]
for this_tbody in tbody:
    # print('loop'+'---'*10)
    for tr in tbody.find_all('tr'):
        row_data = []
        for td in tr.find_all('td'):
            row_data.append(td.text.strip())
        data.append(row_data)
    break

In [ ]:
df = pd.DataFrame(data, columns=['Date', 'Close', 'Open', 'High', 'Low', 'Volume', 'Change'])
df

In [ ]:
def format_data(df):
    data = df.round(3).values.tolist()
    headers = df.columns.tolist()
    table = tabulate(data, headers=headers, tablefmt='grid')
    return table

In [ ]:
format_data(df)

# Selenium

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By

In [ ]:
# driver = webdriver.Chrome()  # Replace with your preferred webdriver

In [ ]:
# url = 'https://hk.investing.com/indices/us-spx-vix-futures-historical-data'
# driver.get(url)
# button = driver.find_element(By.CLASS_NAME, 'relative flex items-center md:gap-6')
# button.click()

In [ ]:
# driver.quit()

In [ ]:
expiry_date_2024 = {
'05': '2024-05-22',
'06': '2024-06-18',
'07': '2024-07-17',
'08': '2024-08-21',
'09': '2024-09-18',
'10': '2024-10-16',
'11': '2024-11-20',
'12': '2024-12-18'}

In [ ]:
import datetime

def get_closest_expiry_date(expiry_date_dict, current_date=None):
    if current_date is None:
        current_date = datetime.datetime.now().date()
    else:
        current_date = datetime.datetime.strptime(current_date, '%Y-%m-%d').date()

    expiry_dates = sorted(expiry_date_dict.values())
    for date in expiry_dates:
        expiry_date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
        if expiry_date >= current_date:
            return expiry_date

    raise ValueError("No closest future expiry date found in the dictionary.")

In [ ]:
expiry_date_2024 = {
    '05': '2024-05-22',
    '06': '2024-06-18',
    '07': '2024-07-17',
    '08': '2024-08-21',
    '09': '2024-09-18',
    '10': '2024-10-16',
    '11': '2024-11-20',
    '12': '2024-12-18'
}

closest_date = get_closest_expiry_date(expiry_date_2024, '2024-07-20')
print(closest_date)  # Output: '2024-05-22'